In [44]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

from nltk.corpus import stopwords
#import nltk
#nltk.download('stopwords')

import os

In [45]:
file_names = ['../data/ley_bases.txt', '../data/constitucion-nacional.txt']

"""if os.path.exists('../data/') and os.path.isdir('../data/'):
    file_names = os.listdir('../data/')

file_names = ['../data/'+file_name for file_name in file_names]

file_names"""

"if os.path.exists('../data/') and os.path.isdir('../data/'):\n    file_names = os.listdir('../data/')\n\nfile_names = ['../data/'+file_name for file_name in file_names]\n\nfile_names"

In [46]:
tokenizer = ByteLevelBPETokenizer()

# ensure tokenizer reserves the special tokens for the vocab
# (using the same as RoBERTa needs)
special_tokens = [
    "<pad>",
    "<unk>",
    "<s>",
    "</s>",
    "<mask>"
]

tokenizer.train(files=file_names,
                vocab_size=30000,
                min_frequency=2,
                special_tokens=special_tokens
                )

Save files to disk

In [47]:
token_dir = '../LawBERTarg/'
if not os.path.exists(token_dir):
    os.makedirs(token_dir)
tokenizer.save_model(token_dir)

['../LawBERTarg/vocab.json', '../LawBERTarg/merges.txt']

## Testing tokenizer

In [55]:
tokenizer = ByteLevelBPETokenizer(vocab = '../LawBERTarg/vocab.json',
                                  merges = '../LawBERTarg/merges.txt',
                                  lowercase = False,)

Example encodings:

In [56]:
print(tokenizer.encode("la ley prohibe a").tokens)

['la', 'Ġley', 'Ġprohib', 'e', 'Ġa']


In [57]:
print(tokenizer.encode("""
ARTÍCULO 30.- Prohibición de imponer restricciones a la importación y a la
exportación. Los VPU adheridos al RIGI podrán importar y exportar libremente
bienes y servicios para la construcción,
""").tokens)

['Ċ', 'ARTÃįCULO', 'Ġ30', '.-', 'ĠPro', 'hibiciÃ³n', 'Ġde', 'Ġimponer', 'Ġrestricciones', 'Ġa', 'Ġla', 'ĠimportaciÃ³n', 'Ġy', 'Ġa', 'Ġla', 'Ċ', 'exportaciÃ³n', '.', 'ĠLos', 'ĠVPU', 'Ġadheridos', 'Ġal', 'ĠRIGI', 'ĠpodrÃ¡n', 'Ġimportar', 'Ġy', 'Ġexportar', 'Ġlibremente', 'Ċ', 'bienes', 'Ġy', 'Ġservicios', 'Ġpara', 'Ġla', 'ĠconstrucciÃ³n', ',', 'Ċ']


Add start and end tokens:

In [58]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)

tokenizer.enable_truncation(max_length=512)

In [59]:
print(tokenizer.encode("legalidad, gestión y auditoría de toda la actividad de la administración pública centralizada y descentralizada").tokens)

['<s>', 'leg', 'alidad', ',', 'ĠgestiÃ³n', 'Ġy', 'ĠauditorÃŃa', 'Ġde', 'Ġtoda', 'Ġla', 'Ġactividad', 'Ġde', 'Ġla', 'ĠadministraciÃ³n', 'ĠpÃºblica', 'Ġcentralizada', 'Ġy', 'Ġdescentralizada', '</s>']


In [60]:
print(tokenizer.encode("Todo Estado parte en el momento de la firma o ratificación de la presente Convención o de su adhesión a la misma, podrá declarar que no se considera obligado por el párrafo 1 del presente artículo.").tokens)

['<s>', 'To', 'do', 'ĠEstado', 'Ġparte', 'Ġen', 'Ġel', 'Ġmomento', 'Ġde', 'Ġla', 'Ġfirma', 'Ġo', 'ĠratificaciÃ³n', 'Ġde', 'Ġla', 'Ġpresente', 'ĠConvenciÃ³n', 'Ġo', 'Ġde', 'Ġsu', 'ĠadhesiÃ³n', 'Ġa', 'Ġla', 'Ġmisma', ',', 'ĠpodrÃ¡', 'Ġdeclarar', 'Ġque', 'Ġno', 'Ġse', 'Ġconsidera', 'Ġobligado', 'Ġpor', 'Ġel', 'ĠpÃ¡rrafo', 'Ġ1', 'Ġdel', 'Ġpresente', 'ĠartÃŃculo', '.', '</s>']


In [61]:
tokenizer.get_vocab_size()

13060